# Fairness  Behind  a  Veil  of  Ignorance

Import necessary dependencies.
Note that the modules "utils_modified" and "funcs_disp_mist_modified" containing some helper functions are adopted from "Fairness Beyond Disparate Treatment & Disparate Impact: Learning Classification without Disparate Mistreatment" by Zafar et. al. (https://github.com/mbilalzafar/fair-classification)

In [1]:
import sys
import numpy as np
import math
import scipy.io
import compas_preprocessing as preproc
import funcs_disp_mist_modified as fdm
from helpers import *
from cvxpy import *
from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit

## Importing the COMPAS dataset

A detailed description of the dataset can be found in the Appendix of the paper

In [4]:
X, y, x_control = preproc.load_compas_data_modified()
sensitive_attrs = list(x_control.keys())

# since we want +1 to be the desired outcome we have to flip our training labels
# previously +1 indicated that a person recidivated within 2 years
y = -y

Looking for file 'compas-scores-two-years.csv' in the current directory...
File found in current directory..

Number of people recidivating within two years
-1    2795
 1    2483
dtype: int64


Features we will be using for classification are: ['intercept', 'age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'race', 'sex', 'priors_count', 'c_charge_degree'] 



Here we compute the maximum distance between any two data points in order to be able to compute the normalized distance

In [5]:
n = X.shape[0]
D = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        D[i,j] = np.linalg.norm(X[i,:]- X[j,:])
d_max = D.max()

Setting some parameters for the solver

In [6]:
max_iters = 100 # for the convex program
max_iter_dccp = 50  # for the dccp algo
tau, mu = 0.005, 1.2 # default dccp parameters, need to be varied per dataset

The next two cells first split the data into train and validation set. Then it first trains an unconstrained classifier and a constrained classifier with specified parameters. This is useful for only running one particular test instead of iterating over alll tested configurations.

In [7]:
rs = ShuffleSplit(n_splits=1, test_size=.1, random_state=None)

for train_index, test_index in rs.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_control_train = {'race':x_control['race'][train_index]}
    x_control_test = {'race':x_control['race'][test_index]}

In [8]:
# set the alpha parameter for our utility function
alpha = -5
# threshold that average utility needs to be above, called tau in the plots  
threshold = -20

num_points, num_features = x_train.shape

# initialize weight vectors to a random value
w_uncons = Variable(num_features)
w_uncons.value = np.random.rand(x_train.shape[1])
w_cons = Variable(num_features)
w_cons.value = np.random.rand(x_train.shape[1])

print("Unconstrained:")

# scale the predictions to be in [-1,1]
y_pred = (x_train*w_uncons)/5

# logistic loss
loss = sum_entries(logistic( mul_elemwise(-y_train, x_train*w_uncons) )  ) / num_points

constraints = [norm(w_uncons) == 1]

prob = Problem(Minimize(loss), constraints)

try:
    prob.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10,
            solver=ECOS, verbose=False,
            max_iters=max_iters, max_iter=max_iter_dccp)
except Exception as e:
    print (e)
    print("check")


# convert solution vector to a numpy array
w_uncons = np.array(w_uncons.value).flatten()

train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test = fdm.get_clf_stats(w_uncons, x_train, y_train, x_control_train, x_test, y_test, x_control_test, sensitive_attrs)

min_util_uncons = eval_min_benefit((np.dot(x_train, w_uncons)/5), y_train)
print("Minimum benefit without constraints: " + str(min_util_uncons) + "\n")
util_uncons = eval_util((np.dot(x_train, w_uncons)/5), y_train,alpha)
print("Utility without constraints: " + str(util_uncons) + "\n")
print("percentage of positively classified: " + str(np.sum(np.sign(np.dot(x_test,w_uncons)))/y_test.shape[0]) + "\n")


print("Constrained:")

# scale the predictions to be in [-1,1]
y_pred = (x_train*w_cons)/5

# logistic loss
loss = sum_entries(logistic( mul_elemwise(-y_train, x_train*w_cons) )  ) / num_points

if math.isinf(alpha):
    min_bene = min_benefit(y_pred, y_train)

    # constraints for our optimization
    constraints = [threshold <= min_bene, norm(w_cons) == 1]
else:
    util = avg_utility(y_pred, y_train, alpha)
    # constraints for our optimization
    constraints = [threshold <= util, norm(w_cons) == 1]

prob = Problem(Minimize(loss), constraints)

solver_failed = False

try:
    prob.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10,
            solver=ECOS, verbose=False,
            max_iters=max_iters, max_iter=max_iter_dccp)
except Exception as e:
    print (e)
    print("check")
    solver_failed = True

# convert solution vector to a numpy array
w_cons = np.array(w_cons.value).flatten()
print(prob.status)

if prob.status != "Converged" and prob.status != "OPTIMAL":
    solver_failed = True
    print("solver failed")
    
if not solver_failed:    
    # achieved utilities of constrained classifier
    util_cons = eval_util((np.dot(x_train, w_cons)/5), y_train, alpha)
    min_bene_cons = eval_min_benefit((np.dot(x_train, w_cons)/5), y_train)

    train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test = fdm.get_clf_stats(w_cons, x_train, y_train, x_control_train, x_test, y_test, x_control_test, sensitive_attrs)
    
    
    print("Min benefit with constraints: " + str(min_bene_cons) + "\n")
    print("Utility with constraints: " + str(util_cons) + "\n")
    print("percentage of positively classified: " + str(np.sum(np.sign(np.dot(x_test,w_cons)))/y_test.shape[0]))

Unconstrained:
[ 1.  1.  1. ... -1.  1.  1.]


Accuracy: 0.642
||  s  || FPR. || FNR. ||
||  0  || 0.37 || 0.35 ||
||  1  || 0.62 || 0.19 ||


Minimum benefit without constraints: 0.1529710778430708

Utility without constraints: -21.74952181979093

percentage of positively classified: 0.21212121212121213

Constrained:
Converged
[ 1.  1.  1. ... -1.  1.  1.]


Accuracy: 0.629
||  s  || FPR. || FNR. ||
||  0  || 0.37 || 0.39 ||
||  1  || 0.60 || 0.21 ||


Min benefit with constraints: 0.22041487821247613

Utility with constraints: -16.405927081450567

percentage of positively classified: 0.17045454545454544


### Generate cross validation splits for both training and evaulation

In [11]:
cross_val_iter = 10
kf = KFold(n_splits=cross_val_iter,shuffle=True)

## Here we train our classifier with different alpha and tau values and save the resulting weight vectors

Note that you need to run this part first before running the evaluation part. Also if the solver fails in one cross validation iteration the value of the weight vector is discarded. However, the computation for that only works after the first cross validation iteration. So if the solver does fail in the first iteration, you get a corresponding error message and need to rerun the training part.

In [12]:
# how many equispaced threshold values to try within the range specified later 
threshold_iter = 20
# from the feasible region we run it for some lower tau values to extend the plots
# of all alpha values to the left such that they all start at the same tau value
# those runs correspond to unconstrained classifiers, they just make the plots look nicer
left_extend_iter = 5

# this holds all the alpha values that we try
alpha_array = [-np.inf,-5,-1,0.1,0.2,0.4,0.8]

# how many different alpha parameter settings we try
no_params = len(alpha_array)

tau_inf = np.concatenate((np.linspace(-23,0.14,left_extend_iter),np.linspace(0.14,0.52,threshold_iter)))
tau_5 = np.concatenate((np.linspace(-23,-23,left_extend_iter),np.linspace(-23,-5.2,threshold_iter)))
tau_1 = np.concatenate((np.linspace(-23,-1.38,left_extend_iter),np.linspace(-1.38,-1.19,threshold_iter)))
tau01 = np.concatenate((np.linspace(-23,0.978,left_extend_iter),np.linspace(0.978,0.9898,threshold_iter)))
tau02 = np.concatenate((np.linspace(-23,0.958,left_extend_iter),np.linspace(0.958,0.981,threshold_iter)))
tau04 = np.concatenate((np.linspace(-23,0.92,left_extend_iter),np.linspace(0.92,0.968,threshold_iter)))
tau08 = np.concatenate((np.linspace(-23,0.88,left_extend_iter),np.linspace(0.88,0.954,threshold_iter)))

# Holds all the tau values that we try depending on alpha
tau_matrix = np.column_stack((tau_inf,tau_5,tau_1,tau01,tau02,tau04,tau08))

# Here we will put the weights resulting from the training of the classifier
weight_matrix = np.zeros((left_extend_iter+threshold_iter,no_params,X.shape[1]))

i = 0
for train_index, test_index in kf.split(X):
    print("#############cross validation iteration " + str(i) + " #########################")
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_control_train = {'race':x_control['race'][train_index]}
    x_control_test = {'race':x_control['race'][test_index]}
    num_points, num_features = x_train.shape

    alpha_counter = 0
    for alpha in alpha_array:
        print("#############Current alpha value: " + str(alpha) + " #########################")

        for j in range(0,left_extend_iter+threshold_iter):
            print("#############Threshold value iteration " + str(j) + " #########################")
            threshold = tau_matrix[j,alpha_counter]
            print("treshold: " + str(threshold))

            w_cons = Variable(num_features)
            w_cons.value = np.random.rand(x_train.shape[1])

            # logistic loss
            loss = sum_entries(logistic( mul_elemwise(-y_train, x_train*w_cons) )  ) / num_points

            # scale the predictions to be in [-1,1]
            y_pred = (x_train*w_cons)/5
            
            if math.isinf(alpha):
                min_bene = min_benefit(y_pred, y_train)

                # constraints for our optimization
                constraints = [threshold <= min_bene, norm(w_cons) == 1]
            else:
                util = avg_utility(y_pred, y_train, alpha)
                # constraints for our optimization
                constraints = [threshold <= util, norm(w_cons) == 1]
            

            prob = Problem(Minimize(loss), constraints)

            solver_failed = False
            
            try:
                prob.solve(method='dccp', tau=tau, mu=mu, tau_max=1e10,
                        solver=ECOS, verbose=False,
                        max_iters=max_iters, max_iter=max_iter_dccp)
            except Exception as e:
                print (e)
                solver_failed = True


            if prob.status != "Converged" and prob.status != "OPTIMAL":
                solver_failed = True

            if not solver_failed: 
                w_cons = np.array(w_cons.value).flatten()

                weight_matrix[j,alpha_counter,:] += w_cons/cross_val_iter

                util_cons = eval_util((np.dot(x_train, w_cons)/5), y_train, alpha)

                train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test = fdm.get_clf_stats(w_cons, x_train, y_train, x_control_train, x_test, y_test, x_control_test, sensitive_attrs)
                print("Utility with constraints: " + str(util_cons) + "\n")
            else:
                # test if all values 0, which they are only in the first cross validation iteration
                if not np.any(weight_matrix[j,alpha_counter,:]):
                    sys.exit("Solver failed in first cross validation iteration")
                else:
                    weight_matrix[j,alpha_counter,:] = weight_matrix[j,alpha_counter,:]*(i + 1)/i
                
        alpha_counter += 1
    i += 1

#############cross validation iteration 0 #########################
#############Current alpha value: -inf #########################
#############Threshold value iteration 0 #########################
treshold: -23.0
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.663
||  s  || FPR. || FNR. ||
||  0  || 0.33 || 0.31 ||
||  1  || 0.64 || 0.18 ||


Utility with constraints: -inf

#############Threshold value iteration 1 #########################
treshold: -17.215
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.663
||  s  || FPR. || FNR. ||
||  0  || 0.33 || 0.31 ||
||  1  || 0.64 || 0.18 ||


Utility with constraints: -inf

#############Threshold value iteration 2 #########################
treshold: -11.43
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.663
||  s  || FPR. || FNR. ||
||  0  || 0.33 || 0.31 ||
||  1  || 0.64 || 0.18 ||


Utility with constraints: -inf

#############Threshold value iteration 3 #########################
treshold: -5.645
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.663
|| 

[ 1. -1. -1. ...  1.  1.  1.]


Accuracy: 0.655
||  s  || FPR. || FNR. ||
||  0  || 0.34 || 0.35 ||
||  1  || 0.61 || 0.17 ||


Utility with constraints: -16.485774689294917

#############Threshold value iteration 8 #########################
treshold: -20.189473684210526
[ 1. -1. -1. ...  1.  1.  1.]


Accuracy: 0.655
||  s  || FPR. || FNR. ||
||  0  || 0.34 || 0.35 ||
||  1  || 0.61 || 0.17 ||


Utility with constraints: -16.467937970538493

#############Threshold value iteration 9 #########################
treshold: -19.25263157894737
[ 1. -1. -1. ...  1.  1.  1.]


Accuracy: 0.657
||  s  || FPR. || FNR. ||
||  0  || 0.34 || 0.35 ||
||  1  || 0.60 || 0.17 ||


Utility with constraints: -16.497738237636046

#############Threshold value iteration 10 #########################
treshold: -18.315789473684212
[ 1. -1. -1. ...  1.  1.  1.]


Accuracy: 0.657
||  s  || FPR. || FNR. ||
||  0  || 0.34 || 0.35 ||
||  1  || 0.60 || 0.17 ||


Utility with constraints: -16.502364165985068

#########

[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.631
||  s  || FPR. || FNR. ||
||  0  || 0.56 || 0.16 ||
||  1  || 0.85 || 0.05 ||


Utility with constraints: -1.2999999999731557

#############Threshold value iteration 14 #########################
treshold: -1.29
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.623
||  s  || FPR. || FNR. ||
||  0  || 0.60 || 0.14 ||
||  1  || 0.87 || 0.05 ||


Utility with constraints: -1.2899999999755127

#############Threshold value iteration 15 #########################
treshold: -1.28
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.608
||  s  || FPR. || FNR. ||
||  0  || 0.65 || 0.13 ||
||  1  || 0.87 || 0.05 ||


Utility with constraints: -1.2799999999750578

#############Threshold value iteration 16 #########################
treshold: -1.27
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.612
||  s  || FPR. || FNR. ||
||  0  || 0.66 || 0.10 ||
||  1  || 0.91 || 0.03 ||


Utility with constraints: -1.26999999997758

#############Threshold value iteration 17 ####################



Accuracy: 0.587
||  s  || FPR. || FNR. ||
||  0  || 0.74 || 0.07 ||
||  1  || 0.96 || 0.02 ||


Utility with constraints: 0.986694737225053

#############Threshold value iteration 20 #########################
treshold: 0.9873157894736843
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.580
||  s  || FPR. || FNR. ||
||  0  || 0.77 || 0.06 ||
||  1  || 0.98 || 0.02 ||


Utility with constraints: 0.9873157901199453

#############Threshold value iteration 21 #########################
treshold: 0.9879368421052631
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.578
||  s  || FPR. || FNR. ||
||  0  || 0.79 || 0.05 ||
||  1  || 0.99 || 0.01 ||


Utility with constraints: 0.9879368424638408

#############Threshold value iteration 22 #########################
treshold: 0.9885578947368421
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.564
||  s  || FPR. || FNR. ||
||  0  || 0.84 || 0.03 ||
||  1  || 1.00 || 0.01 ||


Utility with constraints: 0.9885578949625805

#############Threshold value iteration 23 ####################

[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.663
||  s  || FPR. || FNR. ||
||  0  || 0.33 || 0.31 ||
||  1  || 0.64 || 0.18 ||


Utility with constraints: 0.9260988053944922

#############Threshold value iteration 1 #########################
treshold: -17.02
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.663
||  s  || FPR. || FNR. ||
||  0  || 0.33 || 0.31 ||
||  1  || 0.64 || 0.18 ||


Utility with constraints: 0.9260988054768731

#############Threshold value iteration 2 #########################
treshold: -11.04
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.663
||  s  || FPR. || FNR. ||
||  0  || 0.33 || 0.31 ||
||  1  || 0.64 || 0.18 ||


Utility with constraints: 0.9260988055551541

#############Threshold value iteration 3 #########################
treshold: -5.059999999999999
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.663
||  s  || FPR. || FNR. ||
||  0  || 0.33 || 0.31 ||
||  1  || 0.64 || 0.18 ||


Utility with constraints: 0.9260988055380824

#############Threshold value iteration 4

...

[-1.  1.  1. ...  1.  1.  1.]


Accuracy: 0.600
||  s  || FPR. || FNR. ||
||  0  || 0.72 || 0.10 ||
||  1  || 0.87 || 0.04 ||


Utility with constraints: -inf

#############Threshold value iteration 23 #########################
treshold: 0.5
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.535
||  s  || FPR. || FNR. ||
||  0  || 0.96 || 0.00 ||
||  1  || 0.99 || 0.00 ||


Utility with constraints: -inf

#############Threshold value iteration 24 #########################
treshold: 0.52
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.520
||  s  || FPR. || FNR. ||
||  0  || 1.00 || 0.00 ||
||  1  || 1.00 || 0.00 ||


Utility with constraints: -inf

#############Current alpha value: -5 #########################
#############Threshold value iteration 0 #########################
treshold: -23.0
[-1.  1.  1. ...  1.  1.  1.]


Accuracy: 0.666
||  s  || FPR. || FNR. ||
||  0  || 0.33 || 0.37 ||
||  1  || 0.56 || 0.19 ||


Utility with constraints: -16.29713511978027

#############Threshold value iteration 1 ######

[-1.  1.  1. ...  1.  1.  1.]


Accuracy: 0.687
||  s  || FPR. || FNR. ||
||  0  || 0.30 || 0.34 ||
||  1  || 0.58 || 0.16 ||


Utility with constraints: -1.3782477826745627

#############Threshold value iteration 4 #########################
treshold: -1.38
[-1.  1.  1. ...  1.  1.  1.]


Accuracy: 0.687
||  s  || FPR. || FNR. ||
||  0  || 0.30 || 0.34 ||
||  1  || 0.58 || 0.16 ||


Utility with constraints: -1.3782477869644258

#############Threshold value iteration 5 #########################
treshold: -1.38
[-1.  1.  1. ...  1.  1.  1.]


Accuracy: 0.687
||  s  || FPR. || FNR. ||
||  0  || 0.30 || 0.34 ||
||  1  || 0.58 || 0.16 ||


Utility with constraints: -1.3782477869646554

#############Threshold value iteration 6 #########################
treshold: -1.3699999999999999
[-1.  1.  1. ...  1.  1.  1.]


Accuracy: 0.677
||  s  || FPR. || FNR. ||
||  0  || 0.37 || 0.31 ||
||  1  || 0.60 || 0.13 ||


Utility with constraints: -1.369999999402689

#############Threshold value iteration



Accuracy: 0.668
||  s  || FPR. || FNR. ||
||  0  || 0.45 || 0.23 ||
||  1  || 0.70 || 0.08 ||


Utility with constraints: 0.9804842137316531

#############Threshold value iteration 10 #########################
treshold: 0.9811052631578947
[-1.  1.  1. ...  1.  1.  1.]


Accuracy: 0.651
||  s  || FPR. || FNR. ||
||  0  || 0.49 || 0.22 ||
||  1  || 0.75 || 0.07 ||


Utility with constraints: 0.9811052631838519

#############Threshold value iteration 11 #########################
treshold: 0.9817263157894737
[-1.  1.  1. ...  1.  1.  1.]


Accuracy: 0.647
||  s  || FPR. || FNR. ||
||  0  || 0.52 || 0.19 ||
||  1  || 0.79 || 0.06 ||


Utility with constraints: 0.9817263171680656

#############Threshold value iteration 12 #########################
treshold: 0.9823473684210526
[-1.  1.  1. ...  1.  1.  1.]


Accuracy: 0.641
||  s  || FPR. || FNR. ||
||  0  || 0.56 || 0.17 ||
||  1  || 0.81 || 0.05 ||


Utility with constraints: 0.9823473701982984

#############Threshold value iteration 13 #

[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.626
||  s  || FPR. || FNR. ||
||  0  || 0.64 || 0.11 ||
||  1  || 0.84 || 0.03 ||


Utility with constraints: 0.9701052631688878

#############Threshold value iteration 16 #########################
treshold: 0.9713157894736841
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.619
||  s  || FPR. || FNR. ||
||  0  || 0.67 || 0.10 ||
||  1  || 0.87 || 0.03 ||


Utility with constraints: 0.9713157895306642

#############Threshold value iteration 17 #########################
treshold: 0.9725263157894737
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.603
||  s  || FPR. || FNR. ||
||  0  || 0.71 || 0.09 ||
||  1  || 0.90 || 0.03 ||


Utility with constraints: 0.9725263158306717

#############Threshold value iteration 18 #########################
treshold: 0.9737368421052631
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.598
||  s  || FPR. || FNR. ||
||  0  || 0.72 || 0.09 ||
||  1  || 0.91 || 0.03 ||


Utility with constraints: 0.9737368421879545

#############Threshold value iteratio

[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.562
||  s  || FPR. || FNR. ||
||  0  || 0.86 || 0.05 ||
||  1  || 0.95 || 0.01 ||


Utility with constraints: 0.9604210526443202

#############Threshold value iteration 22 #########################
treshold: 0.9629473684210527
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.560
||  s  || FPR. || FNR. ||
||  0  || 0.88 || 0.04 ||
||  1  || 0.95 || 0.00 ||


Utility with constraints: 0.9629473684279846

#############Threshold value iteration 23 #########################
treshold: 0.9654736842105263
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.550
||  s  || FPR. || FNR. ||
||  0  || 0.91 || 0.01 ||
||  1  || 0.99 || 0.00 ||


Utility with constraints: 0.96547368594361

#############Threshold value iteration 24 #########################
treshold: 0.968
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.522
||  s  || FPR. || FNR. ||
||  0  || 0.99 || 0.01 ||
||  1  || 1.00 || 0.00 ||


Utility with constraints: 0.9680000000547649

#############Current alpha value: 0.8 ##############

### Saving the obtained weight vectors

In [13]:
scipy.io.savemat("../Result_Data/IndividualFairness/W_matrix",{'dummykey':weight_matrix})

In [14]:
w04 = np.transpose(weight_matrix[left_extend_iter:,5,:])
scipy.io.savemat("../Result_Data/IndividualFairness/weight_matrix0-4",{'dummykey':w04})

## Here we evaluate different metrics for all the weight vectors we got from the training process above

Note that we have to use the same validation splits to ensure testing is really done on validation data and not on data already seen during training.

In [15]:
# Here we will put in our results
# one column per parameter setting 
# holds the violations w.r.t. to the dwork constraints
dwork_matrix = np.zeros((left_extend_iter+threshold_iter,no_params))
# holds the false positive rate
fpr_matrix = np.zeros((left_extend_iter+threshold_iter,no_params))
# false negative rate
fnr_matrix = np.zeros((left_extend_iter+threshold_iter,no_params))
# holds the absolute difference in percentage of positive outcomes
dp_matrix = np.zeros((left_extend_iter+threshold_iter,no_params))
# holds the atkinsons index
at_matrix = np.zeros((left_extend_iter+threshold_iter,no_params))
# holds ge2 unfairness
ge_matrix = np.zeros((left_extend_iter+threshold_iter,no_params))
# holds accuracy
accuracy_matrix = np.zeros((left_extend_iter+threshold_iter,no_params))

i = 0
for train_index, test_index in kf.split(X):
    print("#############cross validation iteration " + str(i) + " #########################")
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_control_train = {'race':x_control['race'][train_index]}
    x_control_test = {'race':x_control['race'][test_index]}
    num_points, num_features = x_train.shape
    alpha_counter = 0
    
    for alpha in alpha_array:
        print("#############Current alpha value: " + str(alpha) + " #########################")
        
        for j in range(0,left_extend_iter+threshold_iter):
            print("#############Threshold value iteration " + str(j) + " #########################")
            threshold = tau_matrix[j,alpha_counter]
            print("treshold: " + str(threshold))

            w_cons = weight_matrix[j,alpha_counter,:]

            util_cons = eval_util((np.dot(x_train, w_cons)/5), y_train, alpha)

            train_score, test_score, cov_all_train, cov_all_test, s_attr_to_fp_fn_train, s_attr_to_fp_fn_test = fdm.get_clf_stats(w_cons, x_train, y_train, x_control_train, x_test, y_test, x_control_test, sensitive_attrs)

            y_pred_test = np.sign(np.dot(x_test, w_cons))
            y_pred_train = np.sign(np.dot(x_train, w_cons))

            group_array = x_control_test['race']
            
            # total number of positive and negative points for each group
            unique, counts = np.unique(y_test, return_counts=True)
            n_neg1 = 0
            n_pos1 = 0

            n_neg0 = 0
            n_pos0 = 0
            
            
            # total number of data points in test set
            n = y_test.shape[0]
            unique, counts = np.unique(group_array, return_counts=True)
            # number of people in group 0
            n0 = counts[0]
            # number of people in group 1
            n1 = counts[1]

            # positive prediction count for both classes and for predicted labels
            pos_count0_pred = 0
            pos_count1_pred = 0
            fp_count0 = 0
            fp_count1 = 0
            fn_count0 = 0
            fn_count1 = 0
            
            counter = 0
            for group_membership in group_array:
                if group_membership == 1:
                    if y_test[counter] == 1:
                        n_pos1 += 1
                    if y_test[counter] == -1:
                        n_neg1 += 1
                    # demographic parity check
                    if y_pred_test[counter] == 1:
                        pos_count1_pred += 1
                    # fpr fnr check
                    if y_test[counter] == 1 and y_pred_test[counter] == -1:
                        fn_count1 += 1
                    if y_test[counter] == -1 and y_pred_test[counter] == 1:
                        fp_count1 += 1
                elif group_membership == 0:
                    if y_test[counter] == 1:
                        n_pos0 += 1
                    if y_test[counter] == -1:
                        n_neg0 += 1
                    # demographic parity check
                    if y_pred_test[counter] == 1:
                        pos_count0_pred += 1
                    # fpr fnr check
                    if y_test[counter] == 1 and y_pred_test[counter] == -1:
                        fn_count0 += 1
                    if y_test[counter] == -1 and y_pred_test[counter] == 1:
                        fp_count0 += 1
                else:
                    print("ERROR")
                counter += 1


            print("Utility with constraints: " + str(util_cons) + "\n")
            dwork_matrix[j][alpha_counter] += avg_Dwork_violation(y_pred_test, x_test,0 , d_max) / cross_val_iter
            at_matrix[j][alpha_counter] += atk(y_pred_test, y_test, alpha) / cross_val_iter
            ge_matrix[j][alpha_counter] += ge2(y_pred_test, y_test) / cross_val_iter
            accuracy_matrix[j][alpha_counter] += test_score / cross_val_iter
            fpr_matrix[j][alpha_counter] += (fp_count0/n_neg0 - fp_count1/n_neg1) / cross_val_iter
            fnr_matrix[j][alpha_counter] += (fn_count0/n_pos0 - fn_count1/n_pos1) / cross_val_iter
            dp_matrix[j][alpha_counter] +=  (pos_count0_pred/n0 - pos_count1_pred/n1) / cross_val_iter
        alpha_counter += 1
    i += 1
dp_matrix = np.absolute(dp_matrix)
fpr_matrix = np.absolute(fpr_matrix)
fnr_matrix = np.absolute(fnr_matrix)

#############cross validation iteration 0 #########################
#############Current alpha value: -inf #########################
#############Threshold value iteration 0 #########################
treshold: -23.0
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.674
||  s  || FPR. || FNR. ||
||  0  || 0.34 || 0.31 ||
||  1  || 0.62 || 0.14 ||


Utility with constraints: -inf



./helpers.py:118: RuntimeWarning: divide by zero encountered in power
  return 1 - (np.average(np.power(benefit, alpha))**(1/alpha))/mu


#############Threshold value iteration 1 #########################
treshold: -17.215
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.674
||  s  || FPR. || FNR. ||
||  0  || 0.34 || 0.31 ||
||  1  || 0.62 || 0.14 ||


Utility with constraints: -inf

#############Threshold value iteration 2 #########################
treshold: -11.43
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.674
||  s  || FPR. || FNR. ||
||  0  || 0.34 || 0.31 ||
||  1  || 0.62 || 0.14 ||


Utility with constraints: -inf

#############Threshold value iteration 3 #########################
treshold: -5.645
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.674
||  s  || FPR. || FNR. ||
||  0  || 0.34 || 0.31 ||
||  1  || 0.62 || 0.14 ||


Utility with constraints: -inf

#############Threshold value iteration 4 #########################
treshold: 0.14
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.674
||  s  || FPR. || FNR. ||
||  0  || 0.34 || 0.31 ||
||  1  || 0.62 || 0.14 ||


Utility with constraints: -inf

#############Threshold va

#############Threshold value iteration 9 #########################
treshold: -19.25263157894737
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.672
||  s  || FPR. || FNR. ||
||  0  || 0.34 || 0.33 ||
||  1  || 0.57 || 0.16 ||


Utility with constraints: -16.56055506251016

#############Threshold value iteration 10 #########################
treshold: -18.315789473684212
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.672
||  s  || FPR. || FNR. ||
||  0  || 0.34 || 0.33 ||
||  1  || 0.57 || 0.16 ||


Utility with constraints: -16.57430511891617

#############Threshold value iteration 11 #########################
treshold: -17.378947368421052
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.672
||  s  || FPR. || FNR. ||
||  0  || 0.34 || 0.33 ||
||  1  || 0.57 || 0.16 ||


Utility with constraints: -16.53085904917271

#############Threshold value iteration 12 #########################
treshold: -16.442105263157895
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.667
||  s  || FPR. || FNR. ||
||  0  || 0.35



Accuracy: 0.636
||  s  || FPR. || FNR. ||
||  0  || 0.62 || 0.11 ||
||  1  || 0.91 || 0.03 ||


Utility with constraints: -1.2870261819504856

#############Threshold value iteration 15 #########################
treshold: -1.28
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.634
||  s  || FPR. || FNR. ||
||  0  || 0.65 || 0.09 ||
||  1  || 0.91 || 0.03 ||


Utility with constraints: -1.27704454975145

#############Threshold value iteration 16 #########################
treshold: -1.27
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.631
||  s  || FPR. || FNR. ||
||  0  || 0.68 || 0.08 ||
||  1  || 0.93 || 0.02 ||


Utility with constraints: -1.2670648380012786

#############Threshold value iteration 17 #########################
treshold: -1.26
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.625
||  s  || FPR. || FNR. ||
||  0  || 0.69 || 0.07 ||
||  1  || 0.96 || 0.02 ||


Utility with constraints: -1.257087420121549

#############Threshold value iteration 18 #########################
treshold: -1.25
[1. 1. 1. ... 1

...



Accuracy: 0.662
||  s  || FPR. || FNR. ||
||  0  || 0.39 || 0.26 ||
||  1  || 0.74 || 0.11 ||


Utility with constraints: -inf

#############Threshold value iteration 21 #########################
treshold: 0.46
[ 1. -1. -1. ... -1.  1.  1.]


Accuracy: 0.638
||  s  || FPR. || FNR. ||
||  0  || 0.52 || 0.21 ||
||  1  || 0.81 || 0.07 ||


Utility with constraints: -inf

#############Threshold value iteration 22 #########################
treshold: 0.48000000000000004
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.603
||  s  || FPR. || FNR. ||
||  0  || 0.69 || 0.14 ||
||  1  || 0.94 || 0.02 ||


Utility with constraints: -inf

#############Threshold value iteration 23 #########################
treshold: 0.5
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.558
||  s  || FPR. || FNR. ||
||  0  || 0.96 || 0.00 ||
||  1  || 1.00 || 0.00 ||


Utility with constraints: -inf

#############Threshold value iteration 24 #########################
treshold: 0.52
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.545
||  s  |


Accuracy: 0.706
||  s  || FPR. || FNR. ||
||  0  || 0.26 || 0.31 ||
||  1  || 0.64 || 0.09 ||


Utility with constraints: -1.3765267697535424

#############Threshold value iteration 2 #########################
treshold: -12.19
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.706
||  s  || FPR. || FNR. ||
||  0  || 0.26 || 0.31 ||
||  1  || 0.64 || 0.09 ||


Utility with constraints: -1.376526769863864

#############Threshold value iteration 3 #########################
treshold: -6.785
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.706
||  s  || FPR. || FNR. ||
||  0  || 0.26 || 0.31 ||
||  1  || 0.64 || 0.09 ||


Utility with constraints: -1.376526769883067

#############Threshold value iteration 4 #########################
treshold: -1.38
[ 1. -1.  1. ... -1.  1.  1.]


Accuracy: 0.706
||  s  || FPR. || FNR. ||
||  0  || 0.26 || 0.31 ||
||  1  || 0.64 || 0.09 ||


Utility with constraints: -1.3765267688196738

#############Threshold value iteration 5 #########################
treshold: -1.38

#############Threshold value iteration 8 #########################
treshold: 0.9798631578947369
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.698
||  s  || FPR. || FNR. ||
||  0  || 0.33 || 0.24 ||
||  1  || 0.74 || 0.06 ||


Utility with constraints: 0.9799807113754158

#############Threshold value iteration 9 #########################
treshold: 0.9804842105263157
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.694
||  s  || FPR. || FNR. ||
||  0  || 0.38 || 0.19 ||
||  1  || 0.78 || 0.06 ||


Utility with constraints: 0.9806020909673737

#############Threshold value iteration 10 #########################
treshold: 0.9811052631578947
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.689
||  s  || FPR. || FNR. ||
||  0  || 0.40 || 0.18 ||
||  1  || 0.83 || 0.04 ||


Utility with constraints: 0.9812235380568173

#############Threshold value iteration 11 #########################
treshold: 0.9817263157894737
[ 1. -1.  1. ...  1.  1.  1.]


Accuracy: 0.672
||  s  || FPR. || FNR. ||
||  0  || 0.45 || 

#############Threshold value iteration 14 #########################
treshold: 0.9688947368421053
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.655
||  s  || FPR. || FNR. ||
||  0  || 0.57 || 0.11 ||
||  1  || 0.89 || 0.02 ||


Utility with constraints: 0.9691349639671589

#############Threshold value iteration 15 #########################
treshold: 0.9701052631578947
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.649
||  s  || FPR. || FNR. ||
||  0  || 0.58 || 0.10 ||
||  1  || 0.91 || 0.02 ||


Utility with constraints: 0.9703466440484261

#############Threshold value iteration 16 #########################
treshold: 0.9713157894736841
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.645
||  s  || FPR. || FNR. ||
||  0  || 0.62 || 0.08 ||
||  1  || 0.93 || 0.01 ||


Utility with constraints: 0.9715583812103311

#############Threshold value iteration 17 #########################
treshold: 0.9725263157894737
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.647
||  s  || FPR. || FNR. ||
||  0  || 0.62 || 0.07 ||
||  1  || 0.93

#############Threshold value iteration 20 #########################
treshold: 0.9578947368421052
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.628
||  s  || FPR. || FNR. ||
||  0  || 0.71 || 0.04 ||
||  1  || 0.94 || 0.01 ||


Utility with constraints: 0.9583858502289238

#############Threshold value iteration 21 #########################
treshold: 0.960421052631579
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.602
||  s  || FPR. || FNR. ||
||  0  || 0.78 || 0.04 ||
||  1  || 0.98 || 0.01 ||


Utility with constraints: 0.9609149214251524

#############Threshold value iteration 22 #########################
treshold: 0.9629473684210527
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.590
||  s  || FPR. || FNR. ||
||  0  || 0.84 || 0.03 ||
||  1  || 0.98 || 0.00 ||


Utility with constraints: 0.9634443907085826

#############Threshold value iteration 23 #########################
treshold: 0.9654736842105263
[1. 1. 1. ... 1. 1. 1.]


Accuracy: 0.575
||  s  || FPR. || FNR. ||
||  0  || 0.89 || 0.01 ||
||  1  || 0.99 

### Saving the obtained results

In [16]:
scipy.io.savemat("../Result_Data/IndividualFairness/tau_matrix",{'dummykey':tau_matrix})
scipy.io.savemat("../Result_Data/IndividualFairness/dwork_matrix",{'dummykey':dwork_matrix}) 
scipy.io.savemat("../Result_Data/IndividualFairness/fpr_matrix",{'dummykey':fpr_matrix})
scipy.io.savemat("../Result_Data/IndividualFairness/fnr_matrix",{'dummykey':fnr_matrix})
scipy.io.savemat("../Result_Data/IndividualFairness/dp_matrix",{'dummykey':dp_matrix})
scipy.io.savemat("../Result_Data/IndividualFairness/ge_matrix",{'dummykey':ge_matrix})
scipy.io.savemat("../Result_Data/IndividualFairness/accuracy_matrix",{'dummykey':accuracy_matrix})
scipy.io.savemat("../Result_Data/IndividualFairness/at_matrix",{'dummykey':at_matrix})